In [1]:
import sys
import itertools
import xarray as xr
import numpy as np
import pandas as pd
%matplotlib inline
from functools import partial
# import wrf


# import personal modules
# Path to modules
sys.path.append('../modules')
# Import my modules
from utils import select_months_ds, generate_ptlst_from_start_end
from statistical_tests import ttest_1samp_new

In [2]:
path_to_data = '/expanse/nfs/cw3e/cwp140/'
path_to_out  = '../out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '../figs/'      # figures

In [4]:
region = 'baja' ## 'san_juan' 'baja' 'gulf_of_mexico'
## create bounding boxes for regions
d = {'baja': {'ext': [-115., -110., 27., 32.],
              'start_pt': [33.0, -118.],
              'end_pt': [29.0, -110.]},
     
     'san_juan': {'ext': [-109., -105., 37., 39.],
              'start_pt': [38.0, -112.],
              'end_pt': [38.0, -102.]},

     'gulf_of_mexico': {'ext': [-99., -93., 25., 31.],
              'start_pt': [28.0, -100.],
              'end_pt': [31.0, -93.]}
    }

coord_pairs = generate_ptlst_from_start_end(d[region]['start_pt'][1], d[region]['start_pt'][0], d[region]['end_pt'][1], d[region]['end_pt'][0], pairs=False)
print(coord_pairs)


(array([33.  , 32.75, 32.5 , 32.25, 32.  , 31.75, 31.5 , 31.25, 31.  ,
       30.75, 30.5 , 30.25, 30.  , 29.75, 29.5 , 29.25, 29.  ]), array([-118. , -117.5, -117. , -116.5, -116. , -115.5, -115. , -114.5,
       -114. , -113.5, -113. , -112.5, -112. , -111.5, -111. , -110.5,
       -110. ]))


In [5]:
%%time

def _preprocess(x, coord_pairs):
    lon_pairs = xr.DataArray(coord_pairs[1], dims=['location'])
    lat_pairs = xr.DataArray(coord_pairs[0], dims=['location'])
    return x.sel(longitude=lon_pairs, latitude=lat_pairs, level=slice(1000., 200.))

year = 2000
month = '02'

path_to_data = '/expanse/nfs/cw3e/cwp140/downloads/ERA5/ERA5/{0}/'.format(year)
fname_pattern = "era5_nhemi_025dg_1hr_uvwq_{0}{1}*.nc".format(year, month)

partial_func = partial(_preprocess, coord_pairs=coord_pairs)
ds = xr.open_mfdataset(path_to_data+fname_pattern, engine='netcdf4', combine='by_coords', preprocess=partial_func)  
ds = ds.resample(time='1D').mean()
ds

CPU times: user 1.38 s, sys: 201 ms, total: 1.58 s
Wall time: 11.7 s


<xarray.Dataset>
Dimensions:    (location: 17, level: 23, time: 29)
Coordinates:
    longitude  (location) float32 -118.0 -117.5 -117.0 ... -111.0 -110.5 -110.0
    latitude   (location) float32 33.0 32.75 32.5 32.25 ... 29.5 29.25 29.0
  * level      (level) int32 1000 975 950 925 900 875 ... 350 300 250 225 200
  * time       (time) datetime64[ns] 2000-02-01 2000-02-02 ... 2000-02-29
Dimensions without coordinates: location
Data variables:
    q          (time, level, location) float32 dask.array<chunksize=(1, 23, 17), meta=np.ndarray>
    u          (time, level, location) float32 dask.array<chunksize=(1, 23, 17), meta=np.ndarray>
    v          (time, level, location) float32 dask.array<chunksize=(1, 23, 17), meta=np.ndarray>
    w          (time, level, location) float32 dask.array<chunksize=(1, 23, 17), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2023-12-20 18:33:52 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [6]:
%%time
ds = ds.load()
path_to_out = '/expanse/nfs/cw3e/cwp140/preprocessed/ERA5/cross_section/daily/'
out_fname = "era5_{2}_025dg_daily_uvwq_{0}{1}.nc".format(year, month, region)
ds.to_netcdf(path=path_to_out+out_fname, mode = 'w', format='NETCDF4')

CPU times: user 1min 2s, sys: 40.9 s, total: 1min 43s
Wall time: 23min 27s


In [3]:
## load ar dates within region
region = 'baja' ## 'san_juan' 'baja' 'gulf_of_mexico'
fname = '../out/line_dates_{0}.csv'.format(region)
df = pd.read_csv(fname)
df['day'] = pd.to_datetime(df['datetime']).dt.normalize()
df = df.sort_values(by=['datetime'])
# df = df.set_index(pd.to_datetime(df['datetime'])) ## set daily values as index
ar_dates = df['day'].values
## create year month columns in dataframe 
# df['yearmonth'] = df.index.strftime("%Y%m")
new = df.drop_duplicates('day')
ar_dates = new['day'].values

In [4]:
new

,Unnamed: 0,datetime,day
7273,10,2000-02-23 23:00:00,2000-02-23
7272,9,2000-02-24 00:00:00,2000-02-24
5217,19,2000-04-20 22:00:00,2000-04-20
5215,17,2000-04-21 00:00:00,2000-04-21
5511,34,2000-06-17 01:00:00,2000-06-17
...,...,...,...
6220,567,2023-03-22 05:00:00,2023-03-22
7506,16,2023-06-19 02:00:00,2023-06-19
5372,709,2023-07-31 07:00:00,2023-07-31
5355,692,2023-08-01 00:00:00,2023-08-01


In [5]:
def compute_horizontal_composites(varname, anomaly, ar_dates, ssn, region):
    ## function that computes composites (anomaly or non-anomaly) for defined start_mon and end_mon    
    ## for DJF, MAM, JJA, SON, NDJFMA, and MJJASON
    ## compute anomaly composites - anomaly = True
    ## compute non-anomaly composites - anomaly=False

    ## set start_mon and end_mon based on ssn
    if ssn == 'DJF':
        start_mon, end_mon = (12, 2)
    elif ssn == 'MAM':
        start_mon, end_mon = (3, 5)
    elif ssn == 'JJA':
        start_mon, end_mon = (6, 8)
    elif ssn == 'SON':
        start_mon, end_mon = (9, 11)
    elif ssn == 'NDJFMA':
        start_mon, end_mon = (11, 4)
    elif ssn == 'MJJASO':
        start_mon, end_mon = (5, 10)

    ## load data
    path_to_data = '/expanse/nfs/cw3e/cwp140/downloads/ERA5/'
    out_path = '/home/dnash/DATA/preprocessed/ERA5_composites/'
    
    if anomaly == True:
        fname_pattern = path_to_data + '{0}/anomalies/daily_filtered_anomalies_{0}_*.nc'.format(varname)
    else:
        fname_pattern = path_to_data + '{0}/daily/era5_namerica_025dg_daily_{0}_*.nc'.format(varname)

    ds = xr.open_mfdataset(fname_pattern, engine='netcdf4', combine='by_coords')

    ## subset to AR dates
    ds = ds.sel(time=ar_dates)

    ## subset to start_month and end_month
    ds = select_months_ds(ds, start_mon, end_mon, 'time')
    ds = ds.load()

    ## run students t-test if anomaly == True
    if anomaly == True:    
        popmean = np.zeros([len(ds.latitude), len(ds.longitude)]) ## population mean
        ndays = len(ds.time) # number of unique days
        print(ndays)
        # calculate t-value based on ndays
        a_mean, tval_mask = ttest_1samp_new(a=ds, popmean=popmean, dim='time', n=ndays)

        ## write to netCDF
        out_fname = out_path + '{0}/{2}/filtered_anomaly_composite_{0}_{1}.nc'.format(varname, ssn, region)
        a_mean.to_netcdf(path=out_fname, mode = 'w', format='NETCDF4')

        out_fname = out_path + '{0}/{2}/filtered_anomaly_composite_tvals_{0}_{1}.nc'.format(varname, ssn, region)
        tval_mask.to_netcdf(path=out_fname, mode = 'w', format='NETCDF4')

    else:
        a_mean = ds.mean('time')
        out_fname = out_path + '{0}/{2}/composite_{0}_{1}.nc'.format(varname, ssn, region)
        a_mean.to_netcdf(path=out_fname, mode = 'w', format='NETCDF4')

    return a_mean

In [6]:
%%time
## iterate through options
varname_lst = ['700z', 'ivt']
ssn_lst = ['DJF', 'MAM', 'JJA', 'SON', 'NDJFMA', 'MJJASO']
anom_lst = [True, False]

a = [varname_lst, ssn_lst, anom_lst]

option_lst = list(itertools.product(*a))
for i, lst in enumerate(option_lst):
    anomaly = lst[2]
    ssn = lst[1]
    varname = lst[0]
    print('Anomaly:', anomaly, 'Season:', ssn, 'Variable:', varname)
    tmp = compute_horizontal_composites(varname, anomaly, ar_dates, ssn, region)

Anomaly: True Season: DJF Variable: 700z
1
Anomaly: False Season: DJF Variable: 700z


/home/dnash/miniconda3/envs/SEAK-impacts/lib/python3.12/site-packages/xarray/core/nputils.py:208: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)


Anomaly: True Season: MAM Variable: 700z
56
Anomaly: False Season: MAM Variable: 700z
Anomaly: True Season: JJA Variable: 700z
116
Anomaly: False Season: JJA Variable: 700z
Anomaly: True Season: SON Variable: 700z
52
Anomaly: False Season: SON Variable: 700z
Anomaly: True Season: NDJFMA Variable: 700z
21
Anomaly: False Season: NDJFMA Variable: 700z
Anomaly: True Season: MJJASO Variable: 700z
204
Anomaly: False Season: MJJASO Variable: 700z
Anomaly: True Season: DJF Variable: ivt
1
Anomaly: False Season: DJF Variable: ivt


/home/dnash/miniconda3/envs/SEAK-impacts/lib/python3.12/site-packages/xarray/core/nputils.py:208: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/dnash/miniconda3/envs/SEAK-impacts/lib/python3.12/site-packages/xarray/core/nputils.py:208: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/dnash/miniconda3/envs/SEAK-impacts/lib/python3.12/site-packages/xarray/core/nputils.py:208: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)


Anomaly: True Season: MAM Variable: ivt
56
Anomaly: False Season: MAM Variable: ivt
Anomaly: True Season: JJA Variable: ivt
116
Anomaly: False Season: JJA Variable: ivt
Anomaly: True Season: SON Variable: ivt
52
Anomaly: False Season: SON Variable: ivt
Anomaly: True Season: NDJFMA Variable: ivt
21
Anomaly: False Season: NDJFMA Variable: ivt
Anomaly: True Season: MJJASO Variable: ivt
204
Anomaly: False Season: MJJASO Variable: ivt
CPU times: user 6.9 s, sys: 42.1 s, total: 49 s
Wall time: 4min 8s
